In [27]:
import pickle
import unityneuro.render as urn
import numpy as np
import pandas as pd
from ibllib.atlas import AllenAtlas
atlas = AllenAtlas(25)

# Load data

In [2]:
with open('./data/Block_BWM_2022-06-01.p', 'rb') as file:
    data = pickle.load(file)

In [3]:
data

{'author': 'Brandon Benson',
 'date': '2022-06-01',
 'description': 'Logistic regression is used to decode the block from neural data.  accuracy values are reported after performing regression.  A 5-fold interleaved cross-validation is used to determine hyper-parameters on 80 percent of the data and predictions are made on the held-out 20 percent.  This is repeated 5 times with different held-out sets to cover all trials.  accuracy performance is then evaluated using the predictions across all trials.  The same process is used to decode 100 block pseudo-sessions which are used to evaluate the p-value of the reported accuracy value.',
 'inclusion_crit': 'Sessions must have 400 or more trials.  Trials with reaction time (firstMovement_time-goCue_time) less than 80ms are excluded. Unbiased blocks are excluded.  Regions with less than 10 units are excluded.  QC criteria is 3/3',
 'data':      probe                           session_eid acronym  score_accuracy  \
 0        0  1e45d992-c356-

In [35]:
areas = data['data']

# Setup data for renderer

In [136]:
# load beryl atlas, we only want areas in beryl (and we want *all* areas in beryl)
allenCCF = pd.read_csv('ontology_structure_minimal.csv')
berylIDs = np.unique(allenCCF.beryl_id.values)
berylAcronyms = atlas.regions.id2acronym(berylIDs,mapping='Beryl')
berylAcronyms = np.delete(berylAcronyms, np.argwhere(berylAcronyms=='root')[0][0])

In [137]:
# re-organize so that we have the accuracy/p-value per area
accData = {}
pvalData = {}
for i, row in areas.iterrows():
    if row.acronym in berylAcronyms:
        if row.acronym in accData.keys():
            accData[row.acronym].append(row.score_accuracy)
            pvalData[row.acronym].append(row.pvalue)
        else:
            accData[row.acronym] = [row.score_accuracy]
            pvalData[row.acronym] = [row.pvalue]

In [138]:
# get the median value
areaIntensity = {}
for area in accData.keys():
    areaIntensity[area] = np.median(accData[area])
minAcc = np.min(list(areaIntensity.values()))
maxAcc = np.max(list(areaIntensity.values()))
print(minAcc, maxAcc)
# get the percentage less than 0.05 p-value
pvalProportion = {}
for area in pvalData.keys():
    pvalProportion[area] = len(np.argwhere(np.array(pvalData[area])<0.05))/len(pvalData[area])

0.4076557971683311 0.682917012366165


In [139]:
# rescale intensity
for area in areaIntensity.keys():
    areaIntensity[area] = (areaIntensity[area]-minAcc) / (maxAcc-minAcc) + 0.01

In [140]:
for acronym in berylAcronyms:
    if not acronym in areaIntensity.keys():
        areaIntensity[acronym] = 0
    if not acronym in pvalProportion.keys():
        pvalProportion[acronym] = 0

In [141]:
areaNames = {}
areaMaterials = {}

for acronym in berylAcronyms:
    areaNames[acronym] = True
    areaMaterials[acronym] = "opaque-unlit"

# Render

In [54]:
urn.setup(standalone=True)

(URN) connected to server
Login sent with ID: Dan


In [121]:
urn.clear()

In [155]:
urn.set_area_visibility(areaNames)

In [156]:
urn.set_area_colormap("grey-purple")
urn.set_area_intensity(areaIntensity)

In [149]:
urn.set_area_intensity(pvalProportion)

In [157]:
urn.set_area_intensity(areaIntensity)
urn.set_area_material(areaMaterials)

In [158]:
urn.set_area_colormap("grey-red")
urn.set_area_intensity(pvalProportion)